# C1 — IK Sensível a Energia (Médio)

IK planar 3-DOF: alcançar (x,y) minimizando a mudança em relação ao estado articular anterior (suavidade). Opcionalmente, codifique o custo como uma expectativa variacional.

### Contexto
Braços industriais ou robôs de pernas costumam reutilizar a postura articular anterior como ponto de partida para pequenas correções. Saltos bruscos nos ângulos custam energia, aquecem atuadores e quebram a suavidade do movimento. O **IK sensível a energia** equilibra rastrear o alvo $(x,y)$ e permanecer próximo da configuração articular anterior. [`common.data_utils.energy_aware_ik_dataset`](common/data_utils.py) amostra alvos factíveis e estados prévios para simular correções online.

### Por que quântico?
O objetivo combina cinemática direta não linear com penalidade de suavidade, gerando uma paisagem de perda acidentada. VQE (Variational Quantum Eigensolver) ou atualizações com gradiente natural quântico podem codificar o custo como valor esperado e explorar otimizadores mais ricos sob orçamento apertado de qubits.

## Especificação de Entrada e Saída
**Entrada:** `targets (M×2)`, `prev (M×3)` ângulos articulares anteriores  
**Saída:** `angles (M×3)` ângulos articulares  
Orientações de runtime/recursos: qubits ≤ 10, passos do otimizador ≤ 150; mantenha tempo por amostra ≤ ~0.1 s.

### Dicas de solução
- Custo: $L(\theta) = \lVert f(\theta) - x_{\text{target}} \rVert^2 + \lambda \lVert \theta - \theta_{\text{prev}} \rVert^2$ com comprimentos de elo $(1,1,1)$.
- Codifique $L(\theta)$ como um valor esperado usando [`common.quantum_utils.default_device`](common/quantum_utils.py) e um pequeno ansatz HEA; diferencie com PennyLane para atualizar parâmetros.
- Warm-start com o vetor articular anterior e otimize um residual $\delta \theta$ em vez de ângulos absolutos para manter a busca local.
- Truque híbrido: rode algumas iterações de gradiente clássico e depois entregue a solução a um otimizador quântico (ou vice-versa) para refinamento.

## Setup

In [ ]:
import sys
import os
# Adiciona o diretório pai (qhacka-2025) ao caminho do Python
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import numpy as np
from common import data_utils as du
from common import baselines as bl
from common import quantum_utils as qu
from pennylane import numpy as pnp
import networkx as nx
np.random.seed(1337)

## Baseline (referência)

In [3]:
targets, prev = du.energy_aware_ik_dataset(n=64, seed=0)
import numpy as np
def fk(th):
    L = [1.0, 1.0, 1.0]
    x = sum(L[i]*np.cos(np.sum(th[:i+1])) for i in range(3))
    y = sum(L[i]*np.sin(np.sum(th[:i+1])) for i in range(3))
    return np.array([x, y])
def baseline_energy_aware(target, prev, steps=200, lr=0.03, lam=0.05):
    th = prev.copy()
    for _ in range(steps):
        eps = 1e-3
        g = np.zeros(3)
        for i in range(3):
            e = np.zeros(3); e[i] = eps
            f1 = np.linalg.norm(fk(th+e) - target)**2 + lam*np.linalg.norm((th+e) - prev)**2
            f0 = np.linalg.norm(fk(th) - target)**2 + lam*np.linalg.norm(th - prev)**2
            g[i] = (f1 - f0) / eps
        th -= lr * g
    return th

def baseline_energy_aware_batch(targets, prev, steps=200, lr=0.03, lam=0.05):
    return np.stack([baseline_energy_aware(t, p, steps=steps, lr=lr, lam=lam) for t, p in zip(targets, prev)], axis=0)

angles = baseline_energy_aware_batch(targets, prev)
err = np.mean(np.linalg.norm(np.array([fk(a) for a in angles]) - targets, axis=1))
print("Erro médio de posição do baseline:", err)


Erro médio de posição do baseline: 0.18081451670170162


## Sua tarefa: implementar `solve(...)`

In [ ]:
def solve(targets, prev):
    
    # --- Início da Solução C1 ---
    
    # 1. Imports
    import pennylane as qml
    from pennylane import numpy as pnp
    
    # --- 2. Função 'fk' Clássica (Diferenciável) ---
    def fk(th):
        L = pnp.array([1.0, 1.0, 1.0])
        th_cumulative = pnp.cumsum(th) 
        x = pnp.sum(L * pnp.cos(th_cumulative))
        y = pnp.sum(L * pnp.sin(th_cumulative))
        return pnp.array([x, y])

    # --- 3. Função de Custo Clássica (Diferenciável) ---
    def classical_cost_fn(th, target, prev, lam):
        pos = fk(th)
        pos_error = pnp.sum((pos - target)**2)
        smooth_error = pnp.sum((th - prev)**2)
        return pos_error + lam * smooth_error

    # --- 4. Otimização Principal ---
    M = len(targets)
    angles_out = pnp.zeros((M, 3))
    
    # Nossos parâmetros
    steps = 81
    lr = 0.075
    lam = 0.04
    
    opt = qml.AdamOptimizer(stepsize=lr)

    for i in range(M):
        th = pnp.array(prev[i], requires_grad=True)
        target_i = pnp.array(targets[i])
        
        for _ in range(steps):
            th = opt.step(lambda v: classical_cost_fn(v, target_i, th, lam), th)
            
        angles_out[i, :] = th
            
    return angles_out.unwrap()

# --- Fim da Solução ---

## Testes públicos (rápidos)

In [32]:
targets, prev = du.energy_aware_ik_dataset(n=48, seed=1)
angles = solve(targets, prev)
def fk(th):
    import numpy as np
    L=[1.0,1.0,1.0]
    x = sum(L[i]*np.cos(np.sum(th[:i+1])) for i in range(3))
    y = sum(L[i]*np.sin(np.sum(th[:i+1])) for i in range(3))
    return np.array([x,y])
err = np.mean(np.linalg.norm(np.array([fk(a) for a in angles]) - targets, axis=1))
print("Erro médio de posição (público):", err)
assert angles.shape == prev.shape
assert err <= 0.25, "Erro de IK muito alto"
print("OK")


Erro médio de posição (público): 0.14321584980945173
OK


> Testes adicionais serão executados pelos organizadores com seeds/tamanhos diferentes.